#### Imports

In [1]:
import pandas as pd 
import os

In [2]:
users_prep_df=pd.DataFrame(pd.read_csv('02Prep/usersPrep.csv'))
trainings_prep_df=pd.DataFrame(pd.read_csv('02Prep/trainingsPrep.csv'))
responses_prep_df=pd.DataFrame(pd.read_csv('02Prep/responsesPrep.csv'))
organisation_review_prep_df=pd.DataFrame(pd.read_csv('02Prep/organisationReviewPrep.csv'))

In [3]:
os.makedirs('03Transform',exist_ok=True)

##### 1st KPI (AVG SCORES IN EACH TRAINING && SUCCESS RATE OF EACH TRAINING)

In [4]:
# trainings_prep_df
avg_scores=responses_prep_df.groupby('trainingId')['score'].mean().reset_index().round(2)
avg_scores.columns=['trainingId','avg_scores']
print(avg_scores.sort_values(by='avg_scores',ascending=False))


    trainingId  avg_scores
16          17       57.02
83          84       56.52
38          39       55.66
9           10       55.48
92          93       55.07
..         ...         ...
36          37       45.06
81          82       44.72
79          80       44.65
33          34       43.61
10          11       40.88

[100 rows x 2 columns]


In [5]:
success_rate = responses_prep_df.groupby('trainingId').apply(lambda x: (x['score'] >= 60).mean() * 100).reset_index().round(2)
success_rate.columns = ['trainingId', 'success_rate_percentage']
print(success_rate.sort_values(by='success_rate_percentage',ascending=False))

    trainingId  success_rate_percentage
38          39                    52.04
98          99                    50.60
65          66                    50.56
62          63                    50.54
24          25                    50.52
..         ...                      ...
17          18                    33.00
99         100                    32.41
33          34                    31.82
79          80                    29.20
10          11                    24.51

[100 rows x 2 columns]


C:\Users\RamjeeRai\AppData\Local\Temp\ipykernel_30256\2345469540.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  success_rate = responses_prep_df.groupby('trainingId').apply(lambda x: (x['score'] >= 60).mean() * 100).reset_index().round(2)


In [6]:
avg_success_rate_df=pd.merge(avg_scores, success_rate, on='trainingId')
avg_success_rate_df.to_csv('03Transform/trainingAvgSuccessRateTrans.csv',index=False)

##### 2nd KPI (RESPONSES OF EACH TRAININGS WITH HIGHEST SCORERS OF EACH TRAININGS)

In [7]:
response_count=responses_prep_df.groupby('trainingId').size().reset_index(name='total_responses')
print(response_count)


    trainingId  total_responses
0            1               91
1            2              110
2            3               76
3            4               99
4            5              103
..         ...              ...
95          96              118
96          97               97
97          98               87
98          99               83
99         100              108

[100 rows x 2 columns]


In [8]:
max_scores = responses_prep_df.loc[responses_prep_df.groupby('trainingId')['score'].idxmax()][['trainingId','userId','score']]
max_scores.columns = ['trainingId', 'userId_with_highest_score', 'highest_score']
print(max_scores)


      trainingId  userId_with_highest_score  highest_score
4777           1                        172            100
176            2                        257            100
3564           3                         32             99
2479           4                        287             99
9147           5                        221            100
...          ...                        ...            ...
427           96                        205            100
403           97                         19            100
3994          98                        174            100
2790          99                        139            100
1598         100                        243            100

[100 rows x 3 columns]


In [9]:
(pd.merge(response_count,max_scores, on='trainingId')).to_csv('03Transform/trainingOverviewsTrans.csv')

##### 3rd KPI (TOTAL TRAININGS IN EACH DOMAIN)

In [10]:
# total_trainings_by_domain = trainings_prep_df['domainName'].value_counts().reset_index()
total_trainings_by_domain=trainings_prep_df.groupby(['domainId','domainName']).agg(
    training_count=('trainingId', 'count'),
    earliest_start_date=('startDate', 'min'),
    latest_start_date=('startDate', 'max')
).reset_index()
total_trainings_by_domain.to_csv('03Transform/domainDetailsTrans.csv')


##### 4th KPI(Organisation Review Score)

In [11]:
organisation_review_prep_df['performanceScore']=organisation_review_prep_df['communicationMarks']+organisation_review_prep_df['hardWorkingMarks']+organisation_review_prep_df['disciplineMarks']

In [12]:
# organisation_review_prep_df.drop(columns=['communicationMarks','hardWorkingMarks','disciplineMarks'],axis=1,inplace=True)
organisation_review_prep_df.to_csv('03Transform/organisationReviewTrans.csv',index=False)

##### 5th KPI(Employees Details)

In [13]:
users_prep_df.to_csv('03Transform/usersTrans.csv',index=False)

##### 6th KPI

In [14]:
merged_df = responses_prep_df.merge(organisation_review_prep_df, on=['userId', 'trainingId'], how='left')

# Fill NaN values with the mode of each respective column
merged_df['organisationScore'].fillna(merged_df['organisationScore'].mode()[0], inplace=True)
merged_df['communicationMarks'].fillna(merged_df['communicationMarks'].mode()[0], inplace=True)
merged_df['hardWorkingMarks'].fillna(merged_df['hardWorkingMarks'].mode()[0], inplace=True)
merged_df['disciplineMarks'].fillna(merged_df['disciplineMarks'].mode()[0], inplace=True)
merged_df['performanceScore'].fillna(merged_df['performanceScore'].mode()[0], inplace=True)
# Display the merged DataFrame with filled values
print("Merged DataFrame with NaNs filled:")
print(merged_df)
# merged_df.drop('reviewId',axis=1,inplace=True)


Merged DataFrame with NaNs filled:
       responseId  userId  trainingId  score responseDate  reviewId  \
0               1      95          20      2   2024-10-06       NaN   
1               2     181          28     34   2024-10-07       NaN   
2               3     146           2     71   2024-10-07       NaN   
3               4     293          39      4   2024-10-06       NaN   
4               5       3          86     48   2024-10-07    1655.0   
...           ...     ...         ...    ...          ...       ...   
10503        9997     291           2     46   2024-10-07       NaN   
10504        9998     222           5     46   2024-10-06       NaN   
10505        9999      94          56     10   2024-10-06       NaN   
10506       10000     110          11      7   2024-10-06    4335.0   
10507       10000     110          11      7   2024-10-06    8481.0   

       organisationScore  communicationMarks  hardWorkingMarks  \
0                   89.0                 1.0  

C:\Users\RamjeeRai\AppData\Local\Temp\ipykernel_30256\2250685029.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['organisationScore'].fillna(merged_df['organisationScore'].mode()[0], inplace=True)
C:\Users\RamjeeRai\AppData\Local\Temp\ipykernel_30256\2250685029.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we 

In [15]:
merged_df['performanceScore']=merged_df['communicationMarks']+merged_df['disciplineMarks']+merged_df['hardWorkingMarks']
merged_df['performanceScore'].value_counts()

performanceScore
1.0    8373
2.0    1285
0.0     432
3.0     418
Name: count, dtype: int64

In [16]:
merged_df

,responseId,userId,trainingId,score,responseDate,reviewId,organisationScore,communicationMarks,hardWorkingMarks,disciplineMarks,performanceScore
0,1,95,20,2,2024-10-06,NaN,89.0,1.0,0.0,0.0,1.0
1,2,181,28,34,2024-10-07,NaN,89.0,1.0,0.0,0.0,1.0
2,3,146,2,71,2024-10-07,NaN,89.0,1.0,0.0,0.0,1.0
3,4,293,39,4,2024-10-06,NaN,89.0,1.0,0.0,0.0,1.0
4,5,3,86,48,2024-10-07,1655.0,16.0,1.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
10503,9997,291,2,46,2024-10-07,NaN,89.0,1.0,0.0,0.0,1.0
10504,9998,222,5,46,2024-10-06,NaN,89.0,1.0,0.0,0.0,1.0
10505,9999,94,56,10,2024-10-06,NaN,89.0,1.0,0.0,0.0,1.0
10506,10000,110,11,7,2024-10-06,4335.0,43.0,0.0,1.0,0.0,1.0
